# Amazon SageMaker XGBoost를 활용한 다이렉트 마케팅 타겟팅
_**불균형 클래스를 가진 이진 예측 문제: 그래디언트 부스티드 트리를 활용한 지도 학습**_

---

## 배경
우편, 이메일, 전화 등을 통한 다이렉트 마케팅은 고객을 확보하기 위한 일반적인 전략입니다. 자원과 고객의 관심이 제한되어 있기 때문에, 특정 제안에 반응할 가능성이 높은 잠재 고객 하위 집단만을 대상으로 하는 것이 목표입니다. 인구통계, 과거 상호작용, 환경적 요소와 같은 쉽게 얻을 수 있는 정보를 기반으로 잠재 고객을 예측하는 것은 일반적인 머신 러닝 문제입니다.

이 노트북은 한 번 이상의 전화 통화 후 고객이 은행의 정기 예금에 가입할지 여부를 예측하는 예제 문제를 제시합니다. 단계는 다음과 같습니다:

* Amazon SageMaker 노트북 준비하기
* 인터넷에서 Amazon SageMaker로 데이터 다운로드하기
* Amazon SageMaker 알고리즘에 공급할 수 있도록 데이터 조사 및 변환하기
* 그래디언트 부스팅 알고리즘을 사용하여 모델 추정하기
* 모델의 효과성 평가하기
* 지속적인 예측을 위한 모델 설정하기

---

## 준비

_이 노트북은 ml.m4.xlarge 노트북 인스턴스에서 생성되고 테스트되었습니다._

다음 사항을 지정하는 것부터 시작하겠습니다:

- 훈련 및 모델 데이터에 사용할 S3 버킷과 접두사. 이는 노트북 인스턴스, 훈련 및 호스팅과 동일한 리전에 있어야 합니다.
- 훈련 및 호스팅에 데이터 접근 권한을 부여하는 데 사용되는 IAM 역할 ARN. 이를 생성하는 방법은 문서를 참조하세요. 노트북 인스턴스, 훈련 및/또는 호스팅에 둘 이상의 역할이 필요한 경우, boto 정규식을 적절한 전체 IAM 역할 ARN 문자열로 대체하세요.

In [ ]:
# cell 01
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-dm'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

이제 분석 전반에 걸쳐 사용할 Python 라이브러리들을 가져오겠습니다.

In [ ]:
# cell 02
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions

---

## 데이터
[다이렉트 마케팅 데이터셋](https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip)을 샘플 데이터 s3 버킷에서 다운로드하여 시작하겠습니다.

\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

이제 이것을 Pandas 데이터 프레임으로 읽어서 살펴보겠습니다.

In [ ]:
# cell 04
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

이를 SageMaker Processing으로 처리하기 위해 S3에 네이티브하게 저장하겠습니다.

In [ ]:
# cell 05
from sagemaker import Session

sess = Session()
input_source = sess.upload_data('./bank-additional/bank-additional-full.csv', bucket=bucket, key_prefix=f'{prefix}/input_data')
input_source

# Amazon SageMaker Processing을 이용한 특성 공학

Amazon SageMaker Processing을 사용하면 Amazon SageMaker에서 데이터 전처리 또는 후처리, 특성 공학, 데이터 검증 또는 모델 평가 워크로드를 위한 단계를 실행할 수 있습니다. Processing 작업은 Amazon S3에서 입력으로 데이터를 받아 출력을 Amazon S3에 저장합니다.

![processing](https://sagemaker.readthedocs.io/en/stable/_images/amazon_sagemaker_processing_image1.png)

여기서는 데이터셋을 가져와 SageMaker Processing으로 변환하겠습니다. SageMaker Processing은 노트북 서버를 실행하는 인스턴스와 별도로 SageMaker가 관리하는 클러스터에서 테라바이트 규모의 데이터를 처리하는 데 사용할 수 있습니다. 일반적인 SageMaker 워크플로우에서 노트북은 프로토타이핑에만 사용되며 비교적 저렴하고 성능이 낮은 인스턴스에서 실행할 수 있는 반면, 처리, 훈련 및 모델 호스팅 작업은 별도의 더 강력한 SageMaker 관리 인스턴스에서 실행됩니다. SageMaker Processing은 Scikit-learn을 위한 기본 지원과 함께 Bring Your Own Container 옵션을 제공하므로 다양한 데이터 변환 기술과 작업에 사용할 수 있습니다.

SageMaker Processing을 사용하려면 아래와 같이 Python 데이터 전처리 스크립트를 제공하기만 하면 됩니다. 이 예제에서는 데이터 처리를 위한 많은 일반적인 함수가 포함된 SageMaker 사전 구축된 Scikit-learn 컨테이너를 사용하고 있습니다. 실행할 수 있는 코드와 작업 유형에는 거의 제한이 없으며, 최소한의 계약만 있습니다: 입력 및 출력 데이터는 지정된 디렉토리에 배치되어야 합니다. 이렇게 하면 SageMaker Processing이 자동으로 S3에서 입력 데이터를 로드하고 작업이 완료되면 변환된 데이터를 S3에 다시 업로드합니다.

In [ ]:
%%writefile preprocessing.py
# cell 06


import pandas as pd
import numpy as np
import argparse
import os
from sklearn.preprocessing import OrdinalEncoder

def _parse_args():

    parser = argparse.ArgumentParser()

    # Data, model, and output directories
    # model_dir is always passed in from SageMaker. By default this is a S3 path under the default bucket.
    parser.add_argument('--filepath', type=str, default='/opt/ml/processing/input/')
    parser.add_argument('--filename', type=str, default='bank-additional-full.csv')
    parser.add_argument('--outputpath', type=str, default='/opt/ml/processing/output/')
    parser.add_argument('--categorical_features', type=str, default='y, job, marital, education, default, housing, loan, contact, month, day_of_week, poutcome')

    return parser.parse_known_args()

if __name__=="__main__":
    # Process arguments
    args, _ = _parse_args()
    # Load data
    df = pd.read_csv(os.path.join(args.filepath, args.filename))
    # Change the value . into _
    df = df.replace(regex=r'\.', value='_')
    df = df.replace(regex=r'\_$', value='')
    # Add two new indicators
    df["no_previous_contact"] = (df["pdays"] == 999).astype(int)
    df["not_working"] = df["job"].isin(["student", "retired", "unemployed"]).astype(int)
    df = df.drop(['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'], axis=1)
    # Encode the categorical features
    df = pd.get_dummies(df)
    # Train, test, validation split
    train_data, validation_data, test_data = np.split(df.sample(frac=1, random_state=42), [int(0.7 * len(df)), int(0.9 * len(df))])   # Randomly sort the data then split out first 70%, second 20%, and last 10%
    # Local store
    pd.concat([train_data['y_yes'], train_data.drop(['y_yes','y_no'], axis=1)], axis=1).to_csv(os.path.join(args.outputpath, 'train/train.csv'), index=False, header=False)
    pd.concat([validation_data['y_yes'], validation_data.drop(['y_yes','y_no'], axis=1)], axis=1).to_csv(os.path.join(args.outputpath, 'validation/validation.csv'), index=False, header=False)
    test_data['y_yes'].to_csv(os.path.join(args.outputpath, 'test/test_y.csv'), index=False, header=False)
    test_data.drop(['y_yes','y_no'], axis=1).to_csv(os.path.join(args.outputpath, 'test/test_x.csv'), index=False, header=False)
    print("## Processing complete. Exiting.")

SageMaker Processing 작업을 시작하기 전에, `SKLearnProcessor` 객체를 인스턴스화합니다. 이 객체를 통해 작업에 사용할 인스턴스 유형과 인스턴스 수를 지정할 수 있습니다.

In [ ]:
# cell 07
train_path = f"s3://{bucket}/{prefix}/train"
validation_path = f"s3://{bucket}/{prefix}/validation"
test_path = f"s3://{bucket}/{prefix}/test"

In [ ]:
# cell 08
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role


sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=get_execution_role(),
    instance_type="ml.m5.large",
    instance_count=1, 
    base_job_name='sm-immday-skprocessing'
)

sklearn_processor.run(
    code='preprocessing.py',
    # arguments = ['arg1', 'arg2'],
    inputs=[
        ProcessingInput(
            source=input_source, 
            destination="/opt/ml/processing/input",
            s3_input_mode="File",
            s3_data_distribution_type="ShardedByS3Key"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="train_data", 
            source="/opt/ml/processing/output/train",
            destination=train_path,
        ),
        ProcessingOutput(output_name="validation_data", source="/opt/ml/processing/output/validation", destination=validation_path),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/output/test", destination=test_path),
    ]
)

In [ ]:
# cell 09
!aws s3 ls $train_path/

---

## 실습 1 종료

---

## 훈련
이제 대부분의 특성들이 왜곡된 분포를 가지고 있고, 일부는 서로 높은 상관관계가 있으며, 일부는 목표 변수와 비선형 관계를 가지고 있다는 것을 알게 되었습니다. 또한, 미래의 잠재 고객을 타겟팅하기 위해서는 왜 그 잠재 고객이 타겟팅되었는지 설명하는 것보다 좋은 예측 정확도가 선호됩니다. 이러한 측면들을 종합적으로 고려할 때, 그래디언트 부스팅 트리는 좋은 후보 알고리즘이 됩니다.

알고리즘을 이해하는 데 여러 복잡한 부분이 있지만, 높은 수준에서 그래디언트 부스팅 트리는 이전 모델의 약점을 해결하려고 시도하는 많은 간단한 모델의 예측을 결합하여 작동합니다. 이렇게 함으로써 간단한 모델의 집합이 실제로 크고 복잡한 모델보다 더 나은 성능을 발휘할 수 있습니다. 다른 Amazon SageMaker 노트북에서는 그래디언트 부스팅 트리와 유사한 알고리즘과의 차이점에 대해 더 자세히 설명합니다.

`xgboost`는 그래디언트 부스팅 트리를 위한 매우 인기 있는 오픈 소스 패키지입니다. 이는 계산적으로 강력하고, 기능이 완전하며, 많은 머신 러닝 대회에서 성공적으로 사용되었습니다. Amazon SageMaker의 관리형 분산 훈련 프레임워크를 사용하여 간단한 `xgboost` 모델부터 시작해 보겠습니다.

먼저 Amazon SageMaker의 XGBoost 구현을 위한 ECR 컨테이너 위치를 지정해야 합니다.

In [ ]:
# cell 10
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

그런 다음, CSV 파일 형식으로 훈련하고 있으므로, 훈련 함수가 S3의 파일을 가리키는 포인터로 사용할 수 있는 `s3_input`을 생성하겠습니다. 이는 또한 콘텐츠 타입이 CSV임을 지정합니다.

In [ ]:
# cell 11
s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_path.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=validation_path.format(bucket, prefix), content_type='csv')

먼저 estimator에 훈련 매개변수를 지정해야 합니다. 여기에는 다음이 포함됩니다:
1. `xgboost` 알고리즘 컨테이너
1. 사용할 IAM 역할
1. 훈련 인스턴스 유형 및 수량
1. 출력 데이터를 위한 S3 위치
1. 알고리즘 하이퍼파라미터

그리고 다음을 지정하는 `.fit()` 함수:
1. 출력 데이터를 위한 S3 위치. 이 경우 훈련 세트와 검증 세트가 모두 전달됩니다.

In [ ]:
# cell 12
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---

## 호스팅
이제 `xgboost` 알고리즘을 데이터로 학습시켰으니, 실시간 엔드포인트 뒤에 호스팅되는 모델을 배포해 보겠습니다.

In [ ]:
# cell 13
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

---

## 평가
머신 러닝 모델의 성능을 비교하는 방법은 많지만, 실제 값과 예측 값을 단순히 비교하는 것부터 시작해 보겠습니다. 이 경우, 우리는 고객이 정기 예금에 가입했는지(`1`) 아닌지(`0`)를 예측하는 것이므로 간단한 혼동 행렬(confusion matrix)이 생성됩니다.

먼저 우리의 엔드포인트로 데이터를 어떻게 전달하고 결과를 받을지 결정해야 합니다. 현재 데이터는 노트북 인스턴스의 메모리에 NumPy 배열로 저장되어 있습니다. HTTP POST 요청으로 이를 전송하기 위해, CSV 문자열로 직렬화한 다음 결과 CSV를 디코딩하겠습니다.

*참고: CSV 형식으로 추론할 때, SageMaker XGBoost는 데이터에 타겟 변수가 포함되지 않아야 합니다.*

In [ ]:
# cell 14
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

이제 간단한 함수를 사용하여 다음을 수행하겠습니다:
1. 테스트 데이터셋을 반복합니다
2. 데이터셋을 행 단위의 미니 배치로 분할합니다
3. 이러한 미니 배치를 CSV 문자열 페이로드로 변환합니다(먼저 데이터셋에서 타겟 변수를 제거합니다)
4. XGBoost 엔드포인트를 호출하여 미니 배치 예측을 검색합니다
5. 예측을 수집하고 모델이 제공하는 CSV 출력을 NumPy 배열로 변환합니다

In [ ]:
# cell 15
!aws s3 cp $test_path/test_x.csv /tmp/test_x.csv
!aws s3 cp $test_path/test_y.csv /tmp/test_y.csv

In [ ]:
# cell 16
def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

test_x = pd.read_csv('/tmp/test_x.csv', names=[f'{i}' for i in range(59)])
test_y = pd.read_csv('/tmp/test_y.csv', names=['y'])
predictions = predict(test_x.to_numpy(), xgb_predictor)

이제 예측값과 실제값을 비교하여 혼동 행렬을 확인해보겠습니다.

In [ ]:
# cell 17
pd.crosstab(index=test_y['y'].values, columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

따라서 약 4000명의 잠재 고객 중에서 우리는 136명이 구독할 것이라고 예측했고, 그 중 94명이 실제로 구독했습니다. 또한 우리가 예측하지 못했지만 실제로 구독한 고객이 389명 있었습니다. 이는 바람직하지 않지만, 모델은 이를 개선하기 위해 튜닝될 수 있고(그리고 그래야 합니다). 가장 중요한 점은 최소한의 노력으로 우리의 모델이 [여기](http://media.salford-systems.com/video/tutorial/2015/targeted_marketing.pdf)에 발표된 것과 유사한 정확도를 생성했다는 것입니다.

_알고리즘의 하위 샘플에 일부 무작위 요소가 있기 때문에, 여러분의 결과는 위에 작성된 텍스트와 약간 다를 수 있습니다._

### 정리
이 노트북에서 더 이상 사용하지 않을 리소스를 삭제하세요.

In [ ]:
# cell 18
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

--
## 서버리스 배포 (선택 사항)
모델 학습 후, 모델을 엔드포인트에 배포할 수 있도록 모델 아티팩트를 가져오겠습니다.

In [ ]:
# Setup clients
import boto3

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

In [ ]:
# Retrieve model data from training job
model_artifacts = xgb.model_data
model_artifacts

### 모델 생성
모델 아티팩트, 컨테이너 이미지 URI, 컨테이너용 환경 변수(해당되는 경우), 모델 이름 및 SageMaker IAM 역할을 제공하여 모델을 생성합니다.

In [ ]:
from time import gmtime, strftime

model_name = "xgboost-serverless" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)

# dummy environment variables
byo_container_env_vars = {"SAGEMAKER_CONTAINER_LOG_LEVEL": "20", "SOME_ENV_VAR": "myEnvVar"}

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": container,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": byo_container_env_vars,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response["ModelArn"])

### 엔드포인트 구성 생성
여기서 엔드포인트에 대한 서버리스 구성을 조정할 수 있습니다. 단일 엔드포인트에 대한 현재 최대 동시 호출 수인 MaxConcurrency는 1에서 200 사이의 값이 될 수 있으며, MemorySize는 다음 중 하나가 될 수 있습니다: 1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB 또는 6144 MB.

In [ ]:
xgboost_epc_name = "xgboost-serverless-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=xgboost_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

### 서버리스 엔드포인트 생성
이제 엔드포인트 구성이 완료되었으므로, 서버리스 엔드포인트를 생성하고 모델을 배포할 수 있습니다. 엔드포인트를 생성할 때, 엔드포인트 구성의 이름과 새 엔드포인트의 이름을 제공하세요.

In [ ]:
endpoint_name = "xgboost-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=xgboost_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

엔드포인트 상태가 InService가 될 때까지 기다린 후 엔드포인트를 호출하세요.

In [ ]:
# wait for endpoint to reach a terminal state (InService) using describe endpoint
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

### 엔드포인트 호출
엔드포인트에 요청을 보내 호출합니다. 다음은 공개 Abalone 데이터셋에서 다운로드한 CSV 파일에서 가져온 샘플 데이터 포인트입니다.

In [ ]:
payload = "29,2,999,0,1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0"
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType="text/csv",
)

print(response["Body"].read())

### 정리
이 노트북에서 생성한 리소스 중 더 이상 사용하지 않을 리소스를 삭제하세요.

In [ ]:
client.delete_model(ModelName=model_name)
client.delete_endpoint_config(EndpointConfigName=xgboost_epc_name)
client.delete_endpoint(EndpointName=endpoint_name)

## 자동 모델 튜닝 (선택 사항)
Amazon SageMaker 자동 모델 튜닝(하이퍼파라미터 튜닝이라고도 함)은 사용자가 지정한 알고리즘과 하이퍼파라미터 범위를 사용하여 데이터셋에서 여러 훈련 작업을 실행하여 모델의 최적 버전을 찾습니다. 그런 다음 사용자가 선택한 메트릭에 따라 가장 좋은 성능을 보이는 모델을 만드는 하이퍼파라미터 값을 선택합니다.
예를 들어, 이 마케팅 데이터셋에서 이진 분류 문제를 해결하려고 한다고 가정해 보겠습니다. 목표는 XGBoost 알고리즘 모델을 훈련하여 알고리즘의 곡선 아래 영역(auc) 메트릭을 최대화하는 것입니다. 최상의 모델을 훈련하기 위해 eta, alpha, min_child_weight 및 max_depth 하이퍼파라미터의 어떤 값을 사용해야 하는지 모릅니다. 이러한 하이퍼파라미터의 최적 값을 찾기 위해, Amazon SageMaker 하이퍼파라미터 튜닝이 선택한 목표 메트릭에 따라 가장 좋은 성능을 보이는 훈련 작업을 찾기 위해 검색할 값의 범위를 지정할 수 있습니다. 하이퍼파라미터 튜닝은 지정한 범위 내의 하이퍼파라미터 값을 사용하는 훈련 작업을 시작하고, auc가 가장 높은 훈련 작업을 반환합니다.

In [ ]:
# cell 18
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}


In [ ]:
# cell 19
objective_metric_name = 'validation:auc'

In [ ]:
# cell 20
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)


In [ ]:
# cell 21
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
# cell 22
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
# cell 23
# return the best training job name
tuner.best_training_job()

In [ ]:
# cell 24
#  Deploy the best trained or user specified model to an Amazon SageMaker endpoint
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

In [ ]:
# cell 25
# Create a serializer
tuner_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
# cell 26
# Predict
predictions = predict(test_x.to_numpy(),tuner_predictor)

In [ ]:
# cell 27
# Collect predictions and convert from the CSV output our model provides into a NumPy array
pd.crosstab(index=test_y['y'].values, columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

---

## 확장

이 예제는 비교적 작은 데이터셋을 분석했지만, 분산 및 관리형 훈련과 실시간 모델 호스팅과 같은 Amazon SageMaker 기능을 활용했으며, 이는 훨씬 더 큰 문제에도 쉽게 적용할 수 있습니다. 예측 정확도를 더욱 향상시키기 위해 예측 임계값을 조정하여 거짓 양성과 거짓 음성의 비율을 변경하거나, 하이퍼파라미터 튜닝과 같은 기술을 탐색할 수 있습니다. 실제 시나리오에서는 수작업으로 특성 엔지니어링에 더 많은 시간을 투자하고, 초기 데이터셋에서 사용할 수 없는 고객 정보가 포함된 추가 데이터셋을 찾아볼 가능성이 높습니다.

### 정리

이 노트북을 모두 사용했다면, 아래 셀을 실행해주세요. 이렇게 하면 생성한 호스팅 엔드포인트가 제거되어 인스턴스가 실행된 채로 남아 요금이 발생하는 것을 방지할 수 있습니다.

In [ ]:
# cell 29
tuner_predictor.delete_endpoint(delete_endpoint_config=True)